### Create anndata object from re-mapped E-MTAB-9489 samples (Holloway, 2021, human fetal samples)
- **Developed by:** Anna Maguza
- **Affilation:** Faculty of Medicine, Würzburg University
- **Creation date:** 25th of October 2024
- **Last modified date:** 30th of October 2024

### Import packages

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci
import anndata as ad
from scipy import io,sparse
import os
from datetime import datetime
import muon as mu

+ We want to save the spliced/unspliced counts in our anndata object

In [2]:
def starsolo_velocity_anndata(input_dir):
    """
    input directory should contain barcodes.tsv, features.tsv with 3 mtx from spliced, ambigious, unspliced
    """
    obs = pd.read_csv(os.path.join(input_dir,'barcodes.tsv'), header = None, index_col = 0)
    # Remove index column name to make it compliant with the anndata format
    obs.index.name = None

    var = pd.read_csv(os.path.join(input_dir,"features.tsv"), sep='\t',names = ('gene_ids', 'feature_types'), index_col = 1)
    var.index.name = None

    spliced=sci.sparse.csr_matrix(sci.io.mmread(os.path.join(input_dir,"spliced.mtx")).T)
    ambiguous=sci.sparse.csr_matrix(sci.io.mmread(os.path.join(input_dir,"ambiguous.mtx")).T)
    unspliced=sci.sparse.csr_matrix(sci.io.mmread(os.path.join(input_dir,"unspliced.mtx")).T)
    adata=ad.AnnData(X=spliced,obs=obs,var=var,layers={'spliced':spliced,"ambiguous":ambiguous,"unspliced":unspliced})
    adata.var_names_make_unique()
    return adata

+ Upload sample description dataframe

In [5]:
samples = pd.read_csv("raw_fastq_files/Holloway_2021/E-MTAB-9489/E-MTAB-9489.sdrf.txt", sep = "\t")

In [7]:
samples['sample_name'] = samples['Comment[read2 file]'].str.split('_R2_001.fastq.gz').str[0]

+ Base path for remapped samples

In [10]:
base_path = 'raw_data/Holloway_2021/remapped_fetal_E-MTAB-9489'

In [11]:
ann_data_list = []
failed_samples = []

for sample_name in samples['sample_name']:
    try:
        # Try loading the AnnData object from the UMI10_output path
        sample_path = f"{base_path}/{sample_name}/UMI10_output/Velocyto/raw"
        sample_name_adata = starsolo_velocity_anndata(sample_path)

        # Create the cell_id column
        sample_name_adata.obs['sample_name'] = sample_name

        ann_data_list.append(sample_name_adata)
    except FileNotFoundError:
        try:
            # If not found in UMI10_output, try loading from the UMI12_output path
            sample_path = f"{base_path}/{sample_name}/UMI12_output/Velocyto/raw"
            sample_name_adata = starsolo_velocity_anndata(sample_path)

            # Create the cell_id column
            sample_name_adata.obs['sample_name'] = sample_name

            ann_data_list.append(sample_name_adata)
        except FileNotFoundError:
            # If sample is not found in either path, add it to the failed_samples list
            failed_samples.append(sample_name)
            print(f"Sample {sample_name} not found in both UMI10 and UMI12 paths, skipping.")

# Merge all AnnData objects into one, if there are any
if ann_data_list:
    combined_adata = ann_data_list[0].concatenate(ann_data_list[1:], join='outer')
else:
    combined_adata = None
    print("No valid AnnData objects found to merge.")

# List samples that were not processed
if failed_samples:
    print("The following samples were not processed:")
    for sample in failed_samples:
        print(sample)

/home/amaguza/.local/share/hatch/env/virtual/single-cell-project/HC5eoTg7/single_cell_project/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/amaguza/.local/share/hatch/env/virtual/single-cell-project/HC5eoTg7/single_cell_project/lib/python3.10/site-packages/anndata/utils.py:260: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  warnings.warn(
/home/amaguza/.local/share/hatch/env/virtual/single-cell-project/HC5eoTg7/single_cell_project/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are

In [12]:
combined_adata.obs['barcode'] = combined_adata.obs.index.copy()

In [13]:
combined_adata.obs = combined_adata.obs.merge(samples, on='sample_name', how='left', suffixes=('', '_y'))

combined_adata.obs = combined_adata.obs.loc[:, ~combined_adata.obs.columns.str.endswith('_y')]

In [14]:
combined_adata.obs.index = combined_adata.obs['barcode']

In [15]:
project = 'gut'
species = 'hs'
atribute = 'Holloway2021_E-MTAB-9489'
name = 'AM'
timestamp = datetime.now().strftime('%d%m%Y_%H%M%S')
counts = 'raw'
combined_adata.write_h5ad(f"raw_data/Holloway_2021/{project}_{species}_{atribute}_{name}_{timestamp}_{counts}.h5ad")

In [16]:
mdata = mu.MuData({'rna': combined_adata})
mdata.write(f"raw_data/Holloway_2021/{project}_{species}_{atribute}_{name}_{timestamp}_{counts}.h5mu")

/home/amaguza/.local/share/hatch/env/virtual/single-cell-project/HC5eoTg7/single_cell_project/lib/python3.10/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/amaguza/.local/share/hatch/env/virtual/single-cell-project/HC5eoTg7/single_cell_project/lib/python3.10/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=joi